In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages graphframes:graphframes:0.6.0-spark2.3-s_2.11 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [7]:
from graphframes import * 

vertices = spark.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sqlCtx.createDataFrame([
  ("1", "2", "friend"),
  ("2", "1", "friend"),
  ("1", "3", "friend"),
  ("3", "1", "friend"),
  ("1", "4", "friend"),
  ("4", "1", "friend"),
  ("2", "3", "friend"), 
  ("3", "2", "friend"),
  ("2", "5", "friend"),
  ("5", "2", "friend"),
  ("3", "4", "friend"),
  ("4", "3", "friend"),
  ("3", "5", "friend"),
  ("5", "3", "friend"),
  ("3", "6", "friend"),
  ("6", "3", "friend"),
  ("3", "7", "friend"),
  ("7", "3", "friend")
], ["src", "dst", "relationship"])

gf = GraphFrame(vertices, edges)


### Треугольники

In [4]:
results = gf.triangleCount()
results.show()

+-----+---+-------+---+------+----------+
|count| id|   name|age|gender|university|
+-----+---+-------+---+------+----------+
|    0|  7|   Ilya| 29|     M|       MSU|
|    3|  3|Natasha| 27|     F|     SPbSU|
|    1|  5|   Oleg| 35|     M|      MIPT|
|    0|  6|   Ivan| 30|     M|       MSU|
|    2|  1|   Alex| 28|     M|      MIPT|
|    1|  4|  Pavel| 30|     M|      MIPT|
|    2|  2|  Emeli| 28|     F|      MIPT|
+-----+---+-------+---+------+----------+



### Connected components

In [15]:
vertices = spark.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = spark.createDataFrame([
  ("1", "2", "friend"),
  ("1", "4", "friend"),
  ("3", "5", "friend"),
  ("3", "6", "friend"),
  ("3", "7", "friend")
], ["src", "dst", "type"])

reduced_g = GraphFrame(vertices, edges)

In [13]:
spark.sparkContext.setCheckpointDir("/tmp")
result = reduced_g.connectedComponents()
result.select("id", "component").orderBy("component").show()

+---+-------------+
| id|    component|
+---+-------------+
|  3|  25769803776|
|  7|  25769803776|
|  5|  25769803776|
|  6|  25769803776|
|  2|1236950581248|
|  4|1236950581248|
|  1|1236950581248|
+---+-------------+



### Page Rank

In [14]:
results = gf.pageRank(resetProbability=0.15, maxIter=10)
results.vertices.show()

+---+-------+---+------+----------+------------------+
| id|   name|age|gender|university|          pagerank|
+---+-------+---+------+----------+------------------+
|  1|   Alex| 28|     M|      MIPT| 0.925034321035054|
|  3|Natasha| 27|     F|     SPbSU|1.8587232404145666|
|  2|  Emeli| 28|     F|      MIPT| 0.925034321035054|
|  4|  Pavel| 30|     M|      MIPT|0.6561021410596017|
|  7|   Ilya| 29|     M|       MSU|0.4038005647844105|
|  6|   Ivan| 30|     M|       MSU|0.4038005647844105|
|  5|   Oleg| 35|     M|      MIPT|0.6561021410596017|
+---+-------+---+------+----------+------------------+



In [16]:
results = gf.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.show()

+---+-------+---+------+----------+-------------------+
| id|   name|age|gender|university|           pagerank|
+---+-------+---+------+----------+-------------------+
|  1|   Alex| 28|     M|      MIPT|   1.01935961862719|
|  3|Natasha| 27|     F|     SPbSU| 2.0084281384336697|
|  2|  Emeli| 28|     F|      MIPT|   1.01935961862719|
|  4|  Pavel| 30|     M|      MIPT| 0.7202143452359088|
|  7|   Ilya| 29|     M|       MSU|0.43337602947710424|
|  6|   Ivan| 30|     M|       MSU|0.43337602947710424|
|  5|   Oleg| 35|     M|      MIPT| 0.7202143452359088|
+---+-------+---+------+----------+-------------------+



In [14]:
from pyspark.sql.functions import abs, col, explode, split
from pyspark.sql.types import *

In [ ]:
dataPath = "/share"
graphPath = dataPath + "/LinkPrediction/trainGraph"
usersToPredictPath = dataPath + "/LinkPrediction/prediction.csv"

In [ ]:
data = spark.read.format("csv").option("delimiter", "\t")\
    .load(graphPath).withColumnRenamed("_c0", "user").withColumnRenamed("_c1", "friendsString")

In [ ]:
from pyspark.sql.functions import udf

def cutStartEndBrackets(s):
    return s[2:-2]

cutStartEndBracketsUDF = udf(cutStartEndBrackets, StringType())

userFriend = \
    data.select(col("user"), split(cutStartEndBracketsUDF(col("friendsString")), "\),\(").alias("friendsMasks"))\
    .withColumn("friendMask", explode('friendsMasks'))\
    .withColumn("dst", split(col("friendMask"), ",")[0])\
    .withColumn("src", col("user"))\
    .select(col("src").cast("integer"), col("dst").cast("integer"))

userFriendSymmetric = \
    userFriend\
        .withColumn("tmp", col("src"))\
        .withColumn("src", col("dst"))\
        .withColumn("dst", col("tmp"))\
        .select("src", "dst")
        
edges = userFriend.unionAll(userFriendSymmetric)

vertices = edges.select(col("src").alias("id")).distinct()

In [ ]:
usersFriends = GraphFrame(vertices, edges)

In [ ]:
results = usersFriends.pageRank(resetProbability=0.15, maxIter=10)
